# Machine Learning in Practice

Nora M. Villanueva (nmvillanueva@gradiant.org) & Marta Sestelo (msestelo@gradiant.org)


GDG DevFest Galicia 2018

Santiago de Compostela, Spain. 17th November 2018

## Summary

Machine Learning framework will be introduced by showing both supervised and unsupervised learning methods.  We will describe some of the most well-known and widely used techniques: (1) regression models, (2) classification models, (3) generalized additive models, (4) tree-based models and (5) clustering. We will focus on understanding some mathematical details of these models and how to carry out a practical use of them. We will illustrate these techniques analyzing two different case studies, using to this end a free interactive computing environment, a Jupyter notebook including the R kernel.

----------------




## Table of contents


1. [Linear Regression Models](#lm)
2. [Generalized Linear Models](#glm)
3. [Generalized Additive Models](#gam)
4. [Tree-based Method](#trees)
    - [Decision trees](#tree)
    - [Random Forest](#rf)
5. [Model Evaluation](#eval)
6. [Clustering](#cluster)
 

Before starting, it is necessary to load the required packages. Both `ggplot2`, `dplyr` and `readr` are included in the [`tidyverse`](https://www.tidyverse.org/) package collection, so you can load just this package. 

Note: in the case you use a jupyter notebook you have to load the packages one by one.

The previous packages will be used for a very short exploratory data analysis. For modelling, we are going to use the `mgcv`, `rpart` and `randomForest` packages, apart from the functions included in the `base` package. Finally,  for evaluation, we are going to load the `Metrics`, `OptimalCutpoints` and `ROCR` packages.


In [ ]:
# library(tidyverse)
# For EDA
library(dplyr) 
library(ggplot2)
library(readr)

# For modelling
library(mgcv) 
library(rpart)
library(randomForest)

# More things...
library(rpart.plot)
library(Metrics)
library(OptimalCutpoints)
library(ROCR)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)

<a id="lm"></a>

# Linear Regression

This is one of the most used techniques for assesing the relationship between a set of predictors and the response. There are two main reasons because we want to estimate the coefficients of a linear model: inference and prediction.

The goal of our first use case is to predict the price per square meter of an apartment in Warsaw (Poland) based on selected features such as construction year, surface, floor, number of rooms and district. It should be noted that four of these variables are continuous while the fifth one is a categorical one. Prices are given in Euro. This dataset is available in the `DALEX` package.



In [ ]:
apartments <- read_csv("https://raw.githubusercontent.com/sestelo/machine_learning_workshop_gdg2018/master/apartments.csv")

In [ ]:
dim(apartments)

In [ ]:
summary(apartments)

In [ ]:
apartments$district <- as.factor(apartments$district)

In [ ]:
qplot(y = m2.price, x = district, data = apartments, geom = "boxplot", fill = district)

In [ ]:
qplot(x = surface, y = m2.price, data = apartments)

In [ ]:
model <- lm(m2.price ~ surface, data = apartments) 

In [ ]:
model

In [ ]:
summary(model)

In [ ]:
qplot(x = surface, y = m2.price, data = apartments, geom = "smooth", method = "lm") + geom_point()
# plot(apartments$surface, apartments$m2.price)
# abline(m1)

In [ ]:
lm_model <- lm(m2.price ~ ., data = apartments)

In [ ]:
summary(lm_model)

In [ ]:
levels(apartments$district)

In [ ]:
par(mfrow=c(2,3))
termplot(lm_model, se = TRUE)

In [ ]:
# m3 <- lm(m2.price ~ surface + floor + no.rooms + district, data = apartments)

In [ ]:
# anova(m3, m2) # null model vs full model  test F

In [ ]:
df <- data.frame(construction.year = 2000, surface = 70, floor = 3, no.rooms = 2, district = "Srodmiescie")
muhat <- predict(lm_model, newdata = df)

In [ ]:
muhat

<a id="glm"></a>

# Generalized Linear Models

`R` makes it very easy to fit a logistic regression model. The function used for this purpose is `glm()` and the fitting process is not so different from the one used in linear regression. 

We’ll be working on the Titanic dataset. There are different versions of this dataset freely available online.

The dataset is a collection (n = 1309) of data about some of the passengers and the goal here is to predict the survival (either 1 if the passenger survived or 0 if they did not) based on some features such as the sex, the age, the fare of the ticket, etc. (both categorical and continuous variables).  Data frame with the following 14 columns:

PassengerId: Passenger ID, Survived: Passenger Survival Indicator, Pclass: Passenger Class, Name: Name, Sex: Sex, Age: Age, SibSp: Number of Siblings/Spouses Aboard, Parch: Number of Parents/Children Aboard,Ticket: Ticket Number, Fare: Passenger Fare, Cabin: Cabin, Embarked: Port of Embarkation, Home.dest: Home destination


In [ ]:
download.file("https://goo.gl/At238b", "titanic.csv")

In [ ]:
titanic <- read_csv("titanic.csv")

In [ ]:
dim(titanic)

In [ ]:
head(titanic)

In [ ]:
dim(titanic)

In [ ]:
summary(titanic)

In [ ]:
glm_model <- glm(survived ~ sex + age + fare + sibsp, family = binomial(link = "logit"), data = titanic)

In [ ]:
summary(glm_model)

In [ ]:
# if beta positive, odds ratio > 1: increase prob
# if beta negative, odds ratio < 1: deacrease prob

In [ ]:
exp(coef(glm_model)) # from female to male decreases the probability 92%
                    # to increase a year decreases the probability 2%
                       # to increase 1 euro increases the probability 1%

What will our model tell us about Rose and Jack?

In [ ]:
df <- data.frame(sex = c("female", "male"), age = c(17, 23), fare = c(150, 15), sibsp = c(3, 0))
muhat <- predict(glm_model, newdata = df, type = "response")

In [ ]:
muhat

<a id="gam"></a>

# Generalized Additive Models


Coming back to the `apartments` dataset, can the relationship between the prize of the meter square and each predictor be adequately summarized using a linear equation, or is the relationship more complicated?

Now it is time to fit a model using a more flexible structure. For this, we can apply the `gam()` function of the `mgcv` package which apply splines to estimate each of the f functions. 

In [ ]:
gam_model <- gam(m2.price ~ s(construction.year) + s(surface) + s(floor) + 
             s(no.rooms, k = 5) + district, data = apartments)
# s() Function used in definition of smooth terms within gam model formulae.
# k = the dimension of the basis used to represent the smooth term. 

In [ ]:
summary(gam_model)

In [ ]:
plot(gam_model, pages = 1)

In [ ]:
plot(gam_model, select = 1, shade = TRUE)

In [ ]:
df <- data.frame(construction.year = 2000, surface = 70, floor = 3, no.rooms = 2, district = "Srodmiescie")
muhat <- predict(gam_model, newdata = df)

In [ ]:
print(muhat)

<a id="trees"></a>

# Model based-tree

We move now to the based-tree methods, particularly, we will focus on decision trees, random forest and gradient boosting.

<a id="tree"></a>


## Decision tree 

We start showing how one can apply a decision tree approach. The function `rpart` can run a regression tree if the response variable is numeric, and a classification tree if it is a categorical one. We can see that this technique is a very intuitive and simple approach. Firstly, we carry out the regression analysis taking into account the `apartments` data. 


In [ ]:
tree_model <- rpart(m2.price ~ ., method = "anova", data = apartments) # anova is for regression

In [ ]:
tree_model

In [ ]:
rpart.plot(tree_model)

In [ ]:
df <- data.frame(construction.year = 2000, surface = 70, floor = 3, no.rooms = 2, district = "Srodmiescie")

In [ ]:
predict(tree_model, newdata = df)

<a id="rf"></a>

## Random forest

In this part, we will explain how apply the random forest technique in practice for regresion and classification. 

Now, we will create a Random Forest model with default parameters and then we will fine tune the model by changing `mtry` (number of variables randomly sampled at each stage ) and `ntree` (number of trees). By default, number of trees is 500 and number of variables tried at each split is 1 in this case.

In [ ]:
rf_model <- randomForest(m2.price ~ construction.year + surface + floor + 
                      no.rooms + district, data = apartments, ntree = 500, mtry = 1)
# ntree: number of tree to grow = 1000
# mtry:number of variables randomly sample  = 2  (p/3 - regression, sqrt(p) - classification)

In [ ]:
rf_model

In [ ]:
varImpPlot(rf_model)

In [ ]:
importance(rf_model)

In [ ]:
df <- data.frame(m2.price = 200, construction.year = 2000, surface = 70, floor = 3, no.rooms = 2, 
                 district = "Srodmiescie")
apartments2 <- rbind(apartments, df)

muhat <- predict(rf_model, newdata = apartments2[1001, ])

In [ ]:
muhat

<a id="eval"></a>

# Model Evaluation


Now, we will split the dataset into train and test set in the ratio 80:20. Firstly, we are going to evaluate the regression model and then the classification ones.


## Regression problem

For the evaluation, we are going to use 1-Fold CV with the Mean Square Error. 

In [ ]:
set.seed(30072016)
train <- apartments %>% sample_frac(.80)
test  <- anti_join(apartments, train)


In [ ]:
lm_model <- lm(m2.price ~ ., data = train)
gam_model <- gam(m2.price ~ s(construction.year) + s(surface) + s(floor) + s(no.rooms, k = 5) + district, data = train)
tree_model <- rpart(m2.price ~ ., method = "anova", data = train) 
rf_model <- randomForest(m2.price ~ construction.year + surface 
                         + floor + no.rooms + district, data = train, ntree = 1000, mtry = 2)

muhat_lm <- predict(lm_model, newdata = test)
muhat_gam <- predict(gam_model, newdata = test)
muhat_tree <- predict(tree_model, newdata = test)
muhat_rf <- predict(rf_model, newdata = test)

In [ ]:
(mse_lm <- mean((test$m2.price - muhat_lm)**2))
(mse_gam <- mean((test$m2.price - muhat_gam)**2))
(mse_tree <- mean((test$m2.price - muhat_tree)**2))
(mse_rf <- mean((test$m2.price - muhat_rf)**2))

## Classification problem

For the evaluation, we are going to use 1-Fold CV with the Mean Square Error. 

In [ ]:
set.seed(30072016)
titanic2 <- na.omit(titanic[, c(2, 4, 5, 6, 9)])
titanic2 <- titanic2  %>% mutate (survived = factor(survived), sex = factor(sex))
train <- titanic2 %>% sample_frac(.80)
test  <- anti_join(titanic2, train)


In [ ]:
glm_model <- glm(survived ~ sex + age + fare + sibsp, family = "binomial", data = train)
gam_model <- gam(survived ~ sex + s(age) + s(fare) + s(sibsp, k = 7), family = "binomial", data = train)
tree_model <- rpart(survived ~ sex + age + fare + sibsp, method = "class", data = train)
rf_model <- randomForest(survived ~ sex + age + fare + sibsp, data = train)

muhat_glm <- predict(glm_model, newdata = test, type = "response")
muhat_gam <- predict(gam_model, newdata = test, type = "response")
muhat_tree <- predict(tree_model, newdata = test)
muhat_rf <- predict(rf_model, newdata = test)

For selecting the best model we are going to use the AUC

In [ ]:
(glm_auc <- auc(test$survived, muhat_glm))
(gam_auc <- auc(test$survived, muhat_gam))
(tree_auc <- auc(test$survived, muhat_tree[, 2]))
(rf_auc <- auc(test$survived, as.numeric(muhat_rf)-1))

For calculating the confusion matrix we need to obtain the best threshold....

In [ ]:
aux <- data.frame(muhat = muhat_glm, yreal = test$survived)

opt <- optimal.cutpoints(X = "muhat", status = "yreal", tag.healthy = 0, 
                         methods = "Youden", data = aux, pop.prev = NULL, 
                         control = control.cutpoints(), ci.fit = FALSE, 
                         conf.level = 0.95, trace = FALSE)

In [ ]:
opt

In [ ]:
pred <-ifelse(muhat_glm >= opt$Youden$Global$optimal.cutoff$cutoff, 1, 0)
(confusion_matrix <- table(test$survived, pred))
(accuracy <- (sum(diag(confusion_matrix)))/sum(confusion_matrix))



For plotting the ROC curve...

In [ ]:
pred <- prediction(as.vector(muhat_glm), test$survived)
aux <- performance(pred, "tpr", "fpr")
plot(aux)
mtext(paste("AUC =",round(glm_auc, 4)), side = 1, line =-2, col ="blue" ,cex=0.7)

<a id="cluster"></a>

# Clustering

In [ ]:
head(iris)
x <- iris[,-5]
y <- iris$Species
cl3 <- kmeans(x[, c(1,3)], centers = 3, nstart = 50)


In [ ]:
cl3

In [ ]:
cl3$tot.withinss

In [ ]:
iris  %>% 
    select(Sepal.Length, Petal.Length)  %>% 
    qplot(y = Sepal.Length, x = Petal.Length, data = ., geom = "point", colour = factor(cl3$cluster))

<br/><br/><br/><br/><br/><br/><div style="text-align: right">Material mainly based on the book **[The Elements of Statistical Learning: Data Mining, Inference, and Prediction](https://web.stanford.edu/~hastie/ElemStatLearn/)** (Hastie, Tibshirani and Friedman, 2009).</div>